In [ ]:
import string
import polars as pl

In [ ]:
dummy = pl.from_dict({"text": ["Chinese Beijing Chinese", "Chinese Chinese Shanghai", "Chinese Macao", "Tokyo Japan Chinese"],
    "class": [0, 0, 0, 1]
})

In [ ]:
f_name = "amazon_cells_labelled.txt"
df = pl.read_csv(f_name, has_header=False, sep="\t", new_columns=["text", "class"])
# df = dummy
df = df.with_columns(pl.col("text").str.replace_all("[^\s\w\d]", "").str.to_lowercase())
# print(len(df))
# for r in df.rows():
#     print(r[0])

In [ ]:
df = df.with_columns(pl.col("text").str.replace_all("[^\s\w\d]", "").str.to_lowercase())
df_pos = df.filter(pl.col("class") == 1)
df_neg = df.filter(pl.col("class") == 0)

def count_words(df):
    return df.select(pl.col("text").str.split(" ").alias("words").flatten()).to_series().value_counts()
vocab_size = len(count_words(df))
pos_wc = count_words(df_pos).get_column("counts").sum()
neg_wc = count_words(df_neg).get_column("counts").sum()
print(len(df_pos))

In [ ]:
pos = count_words(df_pos)
def word_dict(df):
    d = {}
    for row in df.rows():
        d[row[0]] = row[1]
    return d
pos_dict = word_dict(count_words(df_pos))
neg_dict = word_dict(count_words(df_neg))

In [ ]:
from math import inf
import typing
VOCAB_SIZE = vocab_size
def cond_prob(word_class_count, word_count):
    return (word_count + 1) / (VOCAB_SIZE + word_class_count)

def sentence_prob(sentence: typing.List[str], class_dict, word_class_count):
    prob = 1.0
    for word in sentence:
        try:
            word_count = class_dict[word]
        except Exception:
            word_count = 0
        cond = cond_prob(word_class_count, word_count)
        print(f"Conditional Prob: {cond}")
        prob *= cond
    return prob

def foo(sentence: typing.List[str], class_dicts: typing.List[typing.Dict], 
    word_class_counts: typing.List[typing.Dict], priors):
    best = -inf
    best_idx = 0
    for idx, (word_class_count, class_dict, prior) in enumerate(zip(word_class_counts, class_dicts, priors)):
        # Todo priors
        prob = prior * sentence_prob(sentence, class_dict, word_class_count)
        print(prob)
        if prob > best:
            best_idx = idx
            best = prob 

    return best_idx, best

foo([s.lower() for s in "worst product ever".split(" ")], [neg_dict, pos_dict], [neg_wc, pos_wc], [2/4, 2/4])

In [ ]:
1/4 * (2/9)**3 * 2/9 * 2/9

In [ ]:
3/4 * (3/7)**3 * 1/14 * 1/14